In [ ]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Agent Framework

This notebook demonstrates how to connect to a remote MCP Server using **Agent Framweork** `HostedMCPTool`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

🔗 [How to use the Model Context Protocol tool with Agent Framework](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-tools?pivots=programming-language-python#mcp-model-context-protocol-tools)


In [1]:

from os import environ
from dotenv import load_dotenv
from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent
from agent_framework import HostedMCPTool
from azure.identity import DefaultAzureCredential
import asyncio

load_dotenv(override=True)

True

In [2]:
mcp_server_url = environ["MCP_SERVER_URL"]
mcp_server_label = environ["MCP_SERVER_LABEL"] 

In [3]:
instructions = """
You are an audit assistant supporting Accounts Payable audit procedures.

You have two authoritative sources, accessed ONLY via tools:
1) Audit Operational Data (MCP/Cosmos): invoices, vendors, payments. This is the ONLY source of truth for operational facts.
2) Audit Policy Library (audit-poc): company audit policy documents (RAG). This is the ONLY source of truth for policy language and criteria. Do not invent policy text or citations if not found.

Rules
- Do not invent records or values. If a tool did not return it, you do not have it.
- Do not use policy text as evidence that a specific invoice/payment violated policy. Use Cosmos records as evidence; use policy snippets as criteria.
- Cite IDs for every operational claim:
  - Invoices → invoice_id(s)
  - Vendors → vendor_id(s)
  - Payments → payment_id(s)
- Cite policy references with policy_id + section_id (or section) + effective_date when available.

Tool selection
- Discovery/listing (Q3 invoices, duplicates, missing PO/receipt, high-risk vendors):
  - Start with query_invoices and/or query_payments, then use get_invoice/get_vendor/get_payment as needed.
- ID-based lookup (exact invoice/vendor/payment ID provided):
  - Use get_invoice, get_vendor, get_payment, or get_payments_for_invoice.
- Policy questions (duplicates, three-way match, vendor governance):
  - Use the policy search/retrieval tools (Azure AI Search) before answering.

Output
- Keep responses audit-ready and structured.
- For findings, include:
  1) Evidence (Cosmos fields + cited IDs)
  2) Policy criteria (policy citation)
  3) Conclusion (clearly labeled; do not overstate certainty)
- If required data is missing, state what is missing and which tool call is needed next.

"""

In [4]:
credential=DefaultAzureCredential()

In [ ]:

async with (AzureAIAgentClient(
        project_endpoint=environ["PROJECT_ENDPOINT"],
        model_deployment_name=environ["MODEL_DEPLOYMENT_NAME"],
        credential=credential,
        agent_name="AuditAgentFramework"
    ).create_agent(
        #instructions=instructions,
        tools=[
    HostedMCPTool(
        name=mcp_server_label,
        url=mcp_server_url,
        approval_mode="never_require"
    )
]
    )   as agent
):
    result = await agent.run("Show me the full details for invoice INV-ANCHOR-001, including any related payments.")
    print(result.text)

I have identified one invoice in Q3 2025. I will now check for potential duplicates where the same vendor and invoice amounts occur within 7 days. Let's process this analysis.There is only one invoice with vendor_id "VEN-1000" and the amount "$4875.33" for the period in Q3 2025. No potential duplicates (with the same vendor_id and amount occurring within 7 days) were found.


In [8]:
# agent = ChatAgent(
#     chat_client=AzureAIAgentClient(credential=credential,
#                                 project_endpoint=environ["PROJECT_ENDPOINT"],
#                                 model_deployment_name=environ["MODEL_DEPLOYMENT_NAME"], 
#                                 instructions=instructions,
#                                    ),
#     instructions=instructions,
#     name="AuditAgentFramework",
#     model_deployment_name=environ["MODEL_DEPLOYMENT_NAME"],
#     tools=[
#         HostedMCPTool(
#             name=mcp_server_label,
#             url=mcp_server_url,
#             approval_mode="never_require"
#         )
#     ]
# )

